# TSP using Genetic algorithm

#### By Niloufar Babaahmadi (610398103)

In [1]:
import numpy as np
import pandas as pd
import more_itertools as mit
import matplotlib as plt
import random
from math import sqrt

First we need to read our input file and create our Matrix accordingly:

In [2]:
def inp_reader(path):
    
    file = open(path)
    inp = file.read()
    file.close()
    
    size = 28
    line = inp.split('\n')
    mat = []
    
    for i in range(8,36):
        
        elements = line[i].split()
        #print(line[i])
        
        for j in range(len(elements)):
            
            elements[j] = int(elements[j])
        
        for h in range(size - len(elements)):
            
            elements.insert(0,0)
        
        mat.append(elements)
        
    return mat

This function will help us find the best and worst solutions in order to find the most eligible paths by calculating the cost of each chromosome.

In [3]:
def fitness_func(chromosome):
    
    path = 0
    #print(size, '\n')
    
    for i in range(len(chromosome)-1):
        
        if chromosome[i] < chromosome[i+1]:
            
            gene1 = chromosome[i]
            gene2 = chromosome[i+1]
            #print(chromosome[i])
            #print(chromosome[i+1], '\n')
            
            
        elif chromosome[i] > chromosome[i+1]:
            
            gene1 = chromosome[i+1]
            gene2 = chromosome[i]
            #print(chromosome[i+1])
            #print(chromosome[i], '\n')
        
        path += mat[gene1][gene2]
        
    if chromosome[0] > chromosome[size-1]:
        
        #print(chromosome[0])
        #print(chromosome[size-1])
        #print(mat[27][9])
        
        
        path += mat[chromosome[0]][chromosome[size-1]]
    
    else:
        
        path += mat[chromosome[size-1]][chromosome[0]]
    
    return path

With the help of this function we can create chromosomes as much as the population.

In [4]:
def society_creator(population):
    
    chromosomes = []
    fitnesses = []
    
    for i in range(population):

        chromosomes.append(mit.random_permutation(range(size)))
        
    for chromosome in chromosomes:
        
        #print(chromosome)
        #print(len(chromosome))
        fitnesses.append(fitness_func(chromosome))
        
    society = pd.DataFrame({'chromosomes':chromosomes,'fitnesses':fitnesses})
    
    #print(society)
    
    return chromosomes, society

Chooses the top best parents for the cross over.

In [5]:
def truncation(T, N):
    
    global society
    #print(society, '\n')
    
    # 0 < T < 1
    society = society.sort_values(by=['fitnesses'])
    int_T = int(T * len(society))
    
    #selecting the first rows of the dataframe
    made_df = society.head(int_T)
    made_df = made_df.sample(N)
    
    return made_df

We choose random variable i1 so that the elements before i1 will be inheritated from gene1 and the elements after i1 will be inheritated from gene2.

In [6]:
def crossover(gene1, gene2):
    
    i1 = np.random.randint(0, size)
    child = []
    
    for i in range(i1):
        
        child.append(gene1[i])
        
    for i in range(i1,size):
        
        if(gene2[i] not in child):
            
            child.append(gene2[i])
            
    #checking if the child has all cities inside
    x = 6 - len(child)
    
    if x!=0 :
        
        for i in range(i1):
            
            if (gene2[i] not in child):
                
                child.append(gene2[i])
                
            if len(child) == size:
                
                break
                
    return child

i1 and i2 are choosen randomly and the elements in between them are shuffled to the given gene.

In [7]:
def mutation(gene):
    
    i1 = np.random.randint(0, size)
    i2 = np.random.randint(i1, size)
    
    helpArr = []
    
    for i in range(i1, i2):
        
        helpArr.append(gene[i])
        
    random.shuffle(helpArr)
    j = 0
    
    for i in range(i1,i2):
        
        gene[i] = helpArr[j]
        j+=1
        
    return gene

In [8]:
global size 
size = 28

global mat
mat = inp_reader("bayg29.tsp")

#print(mat, '\n')

#society_creator(10)
global chromosomes
global society
chromosomes, society = society_creator(5000)
#print(society)

global children

In [9]:
#The algorithm will keep repeating and this iteration is an indicator of that
for i in range(100):
    
    children =[]
    
    #We will choose the top society members as our breeders
    s = truncation(0.3, 1000)['chromosomes'].to_list()
    
    rand1 = 0
    rand2 = 0
    
    #Now children are created
    for i in range(1500):
        
        rand1 = np.random.randint(0, 1000)
        rand2 = np.random.randint(0, 1000)
            
        child = crossover(s[rand1], s[rand2])
        children.append(child)
     
    for i in range(100):
        
        children[i] = mutation(children[i])
        
    fitness = []
    
    #find the fitness of children of the next generation
    for i in range(1500):
        
        fitness.append(fitness_func(children[i]))

    children_dataframe  = pd.DataFrame({'chromosomes':children,'fitnesses':fitness})
    
    #Replace the worst members of the society with children 
    society = society.sort_values(by=['fitnesses'])   
    society = society.head(3500)
    
    frames = [society, children_dataframe]
    society = pd.concat(frames)
    
    last_df = society.sort_values(by=['fitnesses'])
    
    print(last_df)


                                            chromosomes  fitnesses
935   (27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       3063
735   [4, 12, 27, 3, 19, 0, 8, 26, 20, 24, 1, 2, 10,...       3148
453   [21, 6, 8, 27, 25, 7, 1, 24, 2, 5, 10, 3, 0, 1...       3169
560   (11, 4, 7, 0, 19, 12, 14, 16, 21, 8, 27, 6, 20...       3175
844   [2, 7, 5, 24, 18, 23, 26, 25, 15, 16, 17, 27, ...       3218
...                                                 ...        ...
280   [9, 21, 12, 0, 11, 26, 14, 2, 23, 1, 16, 27, 2...       4966
1145  [25, 10, 27, 22, 19, 18, 26, 24, 23, 13, 3, 14...       4968
48    [19, 7, 25, 18, 27, 23, 3, 22, 10, 9, 6, 0, 11...       5049
208   [27, 4, 5, 11, 21, 3, 0, 9, 10, 26, 8, 17, 22,...       5092
83    [18, 21, 14, 27, 1, 24, 16, 6, 22, 7, 9, 5, 20...       5114

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
1128  [6, 3, 23, 26, 24, 20, 13, 1, 25, 15, 21, 5, 1...       2908
998   [27, 8, 24, 2, 6, 23, 26, 5, 10

                                            chromosomes  fitnesses
390   [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2387
649   [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2404
588   [21, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2490
903   [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 11, 9...       2515
662   [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2524
...                                                 ...        ...
80    [21, 16, 20, 12, 4, 23, 9, 5, 2, 13, 26, 10, 2...       4366
46    [4, 24, 25, 6, 0, 22, 27, 23, 5, 14, 15, 17, 2...       4497
1321  [16, 17, 10, 13, 21, 19, 12, 15, 25, 26, 4, 9,...       4502
85    [9, 14, 23, 13, 26, 10, 24, 4, 21, 18, 8, 1, 1...       4648
88    [24, 0, 1, 25, 6, 27, 7, 11, 13, 5, 16, 14, 20...       4687

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
1354  [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2372
976   [15, 25, 27, 20, 6, 23, 26, 5, 

                                            chromosomes  fitnesses
817   [2, 24, 8, 27, 25, 26, 5, 10, 12, 3, 9, 11, 1,...       1934
214   [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2108
439   [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2108
1190  [27, 8, 24, 2, 6, 23, 26, 5, 10, 12, 17, 14, 0...       2108
902   [2, 24, 8, 27, 25, 26, 5, 10, 12, 17, 3, 9, 11...       2128
...                                                 ...        ...
89    [27, 8, 24, 2, 9, 14, 0, 17, 18, 23, 26, 1, 4,...       3936
7     [27, 8, 24, 9, 23, 16, 15, 26, 10, 11, 6, 2, 5...       4076
55    [27, 8, 24, 3, 23, 5, 19, 14, 12, 10, 21, 0, 2...       4109
82    [27, 8, 24, 2, 6, 23, 26, 21, 18, 5, 17, 3, 7,...       4327
34    [27, 8, 20, 15, 0, 9, 1, 17, 5, 19, 2, 18, 13,...       4487

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
817   [2, 24, 8, 27, 25, 26, 5, 10, 12, 3, 9, 11, 1,...       1934
596   [21, 6, 8, 27, 25, 23, 26, 5, 1

                                            chromosomes  fitnesses
1434  [20, 22, 19, 0, 6, 23, 26, 5, 10, 12, 17, 14, ...       1828
732   [20, 22, 19, 0, 6, 23, 26, 5, 10, 12, 17, 14, ...       1828
57    [20, 22, 19, 0, 6, 23, 26, 5, 10, 12, 17, 14, ...       1828
1492  [20, 22, 19, 0, 6, 23, 26, 5, 10, 12, 17, 14, ...       1828
117   [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 3, 9, 11...       1830
...                                                 ...        ...
33    [27, 8, 10, 2, 5, 11, 6, 23, 24, 26, 13, 7, 4,...       4077
76    [27, 8, 24, 2, 6, 3, 7, 9, 5, 23, 1, 10, 12, 1...       4150
85    [27, 20, 23, 11, 10, 15, 4, 21, 5, 16, 14, 26,...       4255
81    [27, 8, 13, 17, 26, 24, 3, 12, 0, 11, 14, 7, 6...       4588
79    [21, 0, 17, 2, 13, 25, 18, 24, 6, 1, 5, 8, 22,...       5168

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
1434  [20, 22, 19, 0, 6, 23, 26, 5, 10, 12, 17, 14, ...       1828
732   [20, 22, 19, 0, 6, 23, 26, 5, 1

                                            chromosomes  fitnesses
1311  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
123   [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
449   [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
1377  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
426   [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
...                                                 ...        ...
30    [2, 24, 19, 20, 12, 22, 9, 8, 25, 18, 13, 16, ...       4170
19    [5, 17, 2, 12, 0, 10, 19, 24, 9, 25, 23, 14, 1...       4200
42    [12, 21, 4, 15, 5, 2, 10, 17, 23, 20, 27, 7, 2...       4233
72    [27, 8, 24, 21, 15, 17, 0, 1, 19, 2, 12, 16, 6...       4311
9     [2, 24, 8, 27, 25, 20, 23, 17, 10, 1, 12, 22, ...       4547

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
1311  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
123   [2, 24, 8, 27, 25, 23, 26, 5, 1

                                            chromosomes  fitnesses
15    [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
201   [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
1260  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
71    [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
1296  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1707
...                                                 ...        ...
99    [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       3351
23    [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       3452
94    [2, 24, 8, 27, 25, 23, 26, 5, 10, 0, 14, 4, 16...       3748
76    [2, 24, 8, 5, 26, 4, 25, 13, 7, 18, 14, 11, 9,...       3807
68    [2, 26, 14, 3, 20, 15, 19, 4, 11, 23, 17, 16, ...       4202

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
15    [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
201   [2, 24, 8, 27, 25, 23, 26, 5, 1

                                            chromosomes  fitnesses
15    [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
1383  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
1382  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
121   [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
1376  [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
...                                                 ...        ...
73    [2, 24, 8, 27, 25, 23, 20, 12, 19, 4, 11, 3, 1...       3527
84    [2, 24, 8, 27, 25, 7, 17, 4, 21, 19, 10, 13, 2...       3588
51    [2, 24, 8, 27, 25, 26, 22, 11, 7, 12, 5, 14, 1...       3923
83    [2, 13, 17, 22, 14, 11, 27, 10, 16, 1, 20, 25,...       4596
27    [3, 4, 5, 27, 19, 24, 22, 17, 26, 10, 13, 7, 8...       4729

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
15    [2, 24, 8, 27, 25, 23, 26, 5, 10, 12, 17, 14, ...       1693
1182  [2, 24, 8, 27, 25, 23, 26, 5, 1

                                            chromosomes  fitnesses
34    [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1685
1443  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1685
116   [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1685
493   [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1685
643   [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1685
...                                                 ...        ...
77    [2, 24, 19, 26, 14, 17, 8, 13, 12, 21, 20, 5, ...       3967
95    [2, 24, 8, 15, 16, 10, 17, 26, 0, 22, 23, 11, ...       3981
64    [2, 24, 19, 16, 20, 11, 10, 4, 26, 21, 14, 5, ...       3998
88    [20, 15, 26, 27, 19, 23, 10, 2, 12, 25, 7, 24,...       4098
60    [22, 8, 27, 12, 14, 4, 15, 13, 3, 0, 24, 26, 7...       4334

[5000 rows x 2 columns]
                                            chromosomes  fitnesses
390   [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1682
34    [2, 24, 8, 27, 25, 23, 26, 5, 1

                                           chromosomes  fitnesses
390  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1682
530  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1682
529  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1682
527  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1682
525  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1682
..                                                 ...        ...
78   [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 21, ...       3623
12   [2, 24, 8, 27, 15, 10, 14, 4, 22, 25, 21, 16, ...       3671
84   [2, 24, 8, 27, 25, 23, 26, 5, 12, 11, 15, 4, 1...       3724
3    [2, 24, 8, 27, 25, 16, 0, 23, 18, 26, 3, 10, 7...       4002
30   [2, 24, 18, 16, 13, 23, 15, 4, 17, 20, 26, 10,...       4225

[5000 rows x 2 columns]
                                           chromosomes  fitnesses
390  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...       1682
311  [2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, 

Now I will print the best chromosome

In [10]:
last_df.head(1)

,chromosomes,fitnesses
390,"[2, 24, 8, 27, 25, 23, 26, 5, 10, 17, 14, 12, ...",1682
